In [1]:
import os
!mkdir dataset
os.chdir('dataset')
!curl -L "https://universe.roboflow.com/ds/PpWAdivglt?key=iyq5E443QL" > badmintonData.zip
!unzip badmintonData.zip && rm badmintonData.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   901  100   901    0     0   1670      0 --:--:-- --:--:-- --:--:--  1668
100 27.3M  100 27.3M    0     0  5648k      0  0:00:04  0:00:04 --:--:-- 8711k
Archive:  badmintonData.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/SAIT_D2_0_187_jpg.rf.df6a5d9d3e3d4214f48f8994205dadac.jpg  
 extracting: test/SAIT_D2_0_234_jpg.rf.a58306f16cf9a9ba61c54aba68c176bf.jpg  
 extracting: test/SAIT_D2_0_289_jpg.rf.0d921269db6bdeb147cccf3513fbbb64.jpg  
 extracting: test/SAIT_D2_0_334_jpg.rf.15719ce4d0d054e17f3b5096cd6d5fd1.jpg  
 extracting: test/SAIT_D2_0_536_jpg.rf.31219756338b37ee77e9c280817827f2.jpg  
 extracting: test/SAIT_D2_0_564_jpg.rf.99f3a2fa26609779041a34673dc8d5e6.jpg  
 extracting: test/SAIT_D2_1_553_jpg.rf.1a00ca95c76460d52cbc488c5d26c696.jpg  
 extracting: te

In [2]:
!pwd

/content/dataset


In [3]:
def create_dir_str(folder,images,classes):
  for i,label in zip(images,classes):
    if label.get('category_id') == 1:
      shutil.move(folder+'/'+i.get('file_name'),folder+'/1')
    if label.get('category_id') == 2:
      shutil.move(folder+'/'+i.get('file_name'),folder+'/2')
    if label.get('category_id') == 3:
      shutil.move(folder+'/'+i.get('file_name'),folder+'/3')
    if label.get('category_id') == 4:
      shutil.move(folder+'/'+i.get('file_name'),folder+'/4')
    if label.get('category_id') == 5:
      shutil.move(folder+'/'+i.get('file_name'),folder+'/5')

In [4]:
!mkdir train/1 train/2 train/3 train/4 train/5
!mkdir valid/1 valid/2 valid/3 valid/4 valid/5
!mkdir test/1 test/2 test/3 test/4 test/5

import json
import shutil

with open('train/_annotations.coco.json', 'r') as f:
    data = json.load(f)
train_images = data.get('images')
train_annotations = data.get('annotations')
create_dir_str('train',train_images, train_annotations)

with open('valid/_annotations.coco.json', 'r') as f:
    data = json.load(f)
valid_images = data.get('images')
valid_annotations = data.get('annotations')
create_dir_str('valid',valid_images,valid_annotations)

with open('test/_annotations.coco.json', 'r') as f:
    data = json.load(f)
test_images = data.get('images')
test_annotations = data.get('annotations')
create_dir_str('test',test_images,test_annotations)


In [5]:
os.chdir('/content/drive/MyDrive/SML_Spring2024/Project_test/dataset/shots')

In [6]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder


vgg16 = models.vgg16(pretrained=True)


num_classes = 5
num_features = vgg16.classifier[-1].in_features
vgg16.classifier[-1] = nn.Linear(num_features, num_classes)


transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


train_dataset = ImageFolder('/content/dataset/train', transform=transform)
val_dataset = ImageFolder('/content/dataset/valid', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(val_dataset, batch_size=32)


criterion = nn.CrossEntropyLoss()


optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)


num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16.to(device)
for epoch in range(num_epochs):
    vgg16.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")


vgg16.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = vgg16(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy on test set: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 92.5MB/s]


Epoch [1/20], Loss: 1.5218
Epoch [2/20], Loss: 1.3250
Epoch [3/20], Loss: 1.1527
Epoch [4/20], Loss: 0.8951
Epoch [5/20], Loss: 0.7389
Epoch [6/20], Loss: 0.4864
Epoch [7/20], Loss: 0.3312
Epoch [8/20], Loss: 0.1850
Epoch [9/20], Loss: 0.0886
Epoch [10/20], Loss: 0.0353
Epoch [11/20], Loss: 0.0225
Epoch [12/20], Loss: 0.0154
Epoch [13/20], Loss: 0.0089
Epoch [14/20], Loss: 0.0588
Epoch [15/20], Loss: 0.0489
Epoch [16/20], Loss: 0.0730
Epoch [17/20], Loss: 0.1169
Epoch [18/20], Loss: 0.0306
Epoch [19/20], Loss: 0.0056
Epoch [20/20], Loss: 0.0014
Accuracy on test set: 0.5484


In [8]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import os

test_dataset = ImageFolder('/content/dataset/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


vgg16.eval()  # Set model to evaluation mode
correct = 0
total = 0

for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = vgg16(inputs)
        _, predicted = torch.max(outputs, 1)
        print('Predicted class: '+str(predicted[0].item())+'\t Ground Truth: '+str(labels[0].item()))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(correct/total)


Predicted class: 0	 Ground Truth: 0
Predicted class: 0	 Ground Truth: 0
Predicted class: 0	 Ground Truth: 0
Predicted class: 0	 Ground Truth: 0
Predicted class: 0	 Ground Truth: 0
Predicted class: 0	 Ground Truth: 0
Predicted class: 1	 Ground Truth: 1
Predicted class: 1	 Ground Truth: 1
Predicted class: 4	 Ground Truth: 1
Predicted class: 1	 Ground Truth: 1
Predicted class: 1	 Ground Truth: 1
Predicted class: 1	 Ground Truth: 1
Predicted class: 1	 Ground Truth: 1
Predicted class: 2	 Ground Truth: 2
Predicted class: 0	 Ground Truth: 2
Predicted class: 2	 Ground Truth: 2
Predicted class: 2	 Ground Truth: 3
Predicted class: 2	 Ground Truth: 4
0.7777777777777778


In [9]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import os


vgg16.eval()  # Set model to evaluation mode

# Preprocess test images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Folder containing test images
folder_path = "/content/drive/MyDrive/SML_Spring2024/Project_SML/Dataset/shots"  # Replace with the path to your test images folder


with open('/content/dataset/test/_annotations.coco.json', 'r') as f:
    data = json.load(f)
categories = data.get('categories')

# Iterate through images in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path)
        image = transform(image)
        image = image.unsqueeze(0)  # Add batch dimension

        # Perform prediction
        with torch.no_grad():
            image = image.cuda()
            output = vgg16(image)
            _, predicted = torch.max(output, 1)
            # If you have class labels, you can map the predicted index to class label
            # predicted_label = class_labels[predicted.item()]
            # print(f"Image: {filename}, Predicted class index: {predicted.item()}")
            typeOfShot = categories[int(predicted.item())].get('name')
            print(f"Image: {filename}, Predicted class: {typeOfShot}")

    else:
        continue




Image: smash1.jpg, Predicted class: Drop Shot
Image: smash2.jpg, Predicted class: Over defensive clear
Image: smash3.jpg, Predicted class: shot-type
Image: smash4.jpg, Predicted class: Serve
Image: smash5.jpg, Predicted class: Net shot
Image: lift1.jpg, Predicted class: shot-type
Image: lift2.jpg, Predicted class: Over defensive clear
Image: lift3.jpg, Predicted class: Serve
Image: lift4.jpg, Predicted class: shot-type
Image: lift5.jpg, Predicted class: Over defensive clear
Image: netshot1.jpg, Predicted class: shot-type
Image: netshot2.jpg, Predicted class: shot-type
Image: netshot3.jpg, Predicted class: shot-type
Image: netshot4.jpg, Predicted class: shot-type
Image: netshot5.jpg, Predicted class: Over defensive clear
